In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv

In [9]:
load_dotenv(dotenv_path=find_dotenv())
from financial.model import Model
import financial.data as fd
from financial.io.file.cache import FileCache

In [10]:
cache = os.environ["CACHE"]
model = os.environ["MODEL"]
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache+"/"))

In [14]:
model_svr = Model.from_file('global_svr_first_experiment', path = os.environ["MODEL"])
predictions_svr = model_svr.get_data(datastore, start_index="2010-01-01", end_index="2023-12-31")
print(predictions_svr)

2010-05-13   -0.016935
2010-05-14   -0.024465
2010-05-17   -0.018561
2010-05-18   -0.020164
2010-05-19   -0.024214
                ...   
2023-12-22    0.007597
2023-12-26    0.010535
2023-12-27    0.017839
2023-12-28    0.022443
2023-12-29    0.028124
Length: 3432, dtype: float64


/home/manidmt/Universidad/TFG/OTRI/my_env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(


In [15]:
model_rforest = Model.from_file('global_forest_first_experiment', path = os.environ["MODEL"])
predictions_rforest = model_rforest.get_data(datastore, start_index="2010-01-01", end_index="2023-12-31")
print(predictions_rforest)

2010-05-13    0.042892
2010-05-14    0.027870
2010-05-17    0.016233
2010-05-18    0.005841
2010-05-19    0.000715
                ...   
2023-12-22   -0.035808
2023-12-26   -0.031003
2023-12-27   -0.033587
2023-12-28   -0.031352
2023-12-29   -0.033377
Length: 3432, dtype: float64


/home/manidmt/Universidad/TFG/OTRI/my_env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
